In [37]:
import torch
from torch import nn
from torch.utils.data import DataLoader, dataloader
from torchvision import datasets
from torchvision.transforms import ToTensor

### 一、下载训练数据，从开源数据上下载

In [38]:
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=ToTensor())

In [39]:
batch_size = 64

# 创建 data_loader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [40]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using Device: {device}")

# define Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(True),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

using Device: cuda
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [41]:
# 定义损失函数
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [42]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Loss: {loss:7f} [{current:>5d}/{size:>5d}]")

In [43]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(dim=1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \nAverage loss: {(100*correct):>0.1f}%, Avg loss: {(test_loss):>8f}\n")

In [44]:
epochs = 5
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch+1, epochs))
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1/5
Loss: 2.298065 [   64/60000]
Loss: 2.284890 [ 6464/60000]
Loss: 2.267618 [12864/60000]
Loss: 2.262902 [19264/60000]
Loss: 2.240703 [25664/60000]
Loss: 2.207161 [32064/60000]
Loss: 2.222152 [38464/60000]
Loss: 2.176962 [44864/60000]
Loss: 2.182051 [51264/60000]
Loss: 2.161593 [57664/60000]
Test Error: 
Average loss: 47.3%, Avg loss: 2.142930

Epoch 2/5
Loss: 2.146550 [   64/60000]
Loss: 2.135390 [ 6464/60000]
Loss: 2.080683 [12864/60000]
Loss: 2.104974 [19264/60000]
Loss: 2.038049 [25664/60000]
Loss: 1.978109 [32064/60000]
Loss: 2.012372 [38464/60000]
Loss: 1.921444 [44864/60000]
Loss: 1.941307 [51264/60000]
Loss: 1.878718 [57664/60000]
Test Error: 
Average loss: 56.2%, Avg loss: 1.865853

Epoch 3/5
Loss: 1.889373 [   64/60000]
Loss: 1.861230 [ 6464/60000]
Loss: 1.749989 [12864/60000]
Loss: 1.798509 [19264/60000]
Loss: 1.670526 [25664/60000]
Loss: 1.628932 [32064/60000]
Loss: 1.653092 [38464/60000]
Loss: 1.548115 [44864/60000]
Loss: 1.592364 [51264/60000]
Loss: 1.483007 [57664